# PCA
on same 4 models

In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess as sp

In [2]:
import json
with open('/home/nick/anaconda3/bin/Tirocinio/doc2vec_tryouts/trend_analisys.json', 'r') as file:
    docs = json.load(file)
print("Length of docs:",len(docs))

# search for duplicates
import utils
docs = utils.delete_duplicates_from_list(docs)
len(docs)

Length of docs: 293


293

In [3]:
import multiprocessing
cores = multiprocessing.cpu_count()
# train the models 
train_corpus = [TaggedDocument(sp(doc['title']+doc['abstract']), [i]) for i, doc in enumerate(docs)]
print("Number of docs:",len(train_corpus))
models = [
    Doc2Vec(vector_size = 35,
                alpha= 0.030,
                min_alpha=0.00030,
                min_count=2, # words that appear less than twice in the corpus are ignored
                dm=0, comment='first model, 35 features') ,
    Doc2Vec(dm=0, vector_size= 50, window=10, negative=5, hs=0, min_count=2, sample=0,
            epochs= 10, workers=cores, alpha= 0.05, comment='alpha=0.05, dm=1, vec_size=40'),
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs= 40, workers=cores, comment='dm=0, vec_size=100'), 
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=3, sample=0,
             epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800, min_count=3')
]

for model in models:
    model.build_vocab(train_corpus)
    print(model, "vocabulary built, with size:", len(model.wv.vocab))
model = models[0]
for epoch in range(300):
    if(epoch%50==0):
        print('iteration {0}'.format(epoch))
    model.train(train_corpus,
                total_examples = model.corpus_count,
                epochs = model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

for i, model in enumerate(models):
    if i == 0:
        continue
    print("Training %s" % model)
    %time model.train(train_corpus, total_examples=len(train_corpus), epochs=model.epochs)
    

Number of docs: 293
Doc2Vec("first model, 35 features",dbow,d35,n5,mc2,s0.001,t3) vocabulary built, with size: 7507
Doc2Vec("alpha=0.05, dm=1, vec_size=40",dbow,d50,n5,mc2,t4) vocabulary built, with size: 7507
Doc2Vec("dm=0, vec_size=100",dbow,d100,n5,mc2,t4) vocabulary built, with size: 7507
Doc2Vec("dm=0, vec = 800, min_count=3",dbow,d800,n5,mc3,t4) vocabulary built, with size: 4747
iteration 0
iteration 50
iteration 100
iteration 150
iteration 200
iteration 250
Training Doc2Vec("alpha=0.05, dm=1, vec_size=40",dbow,d50,n5,mc2,t4)
CPU times: user 2.2 s, sys: 24 ms, total: 2.22 s
Wall time: 773 ms
Training Doc2Vec("dm=0, vec_size=100",dbow,d100,n5,mc2,t4)
CPU times: user 9.26 s, sys: 48 ms, total: 9.31 s
Wall time: 3.14 s
Training Doc2Vec("dm=0, vec = 800, min_count=3",dbow,d800,n5,mc3,t4)
CPU times: user 35.5 s, sys: 232 ms, total: 35.7 s
Wall time: 10.7 s


In [10]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.graph_objs as go

tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
descr = [m.comment for m in models]

from sklearn.decomposition import PCA

def visualize_pca(model, subsample_length, titles):
    # get list of doc_vecs
    doc_vecs = [model[i] for i in range(len(model.docvecs))]
    # apply pca
    pcomponents = PCA(n_components = 2).fit_transform(doc_vecs)
    
    # now return plotly data over a subsample
    colors = ['blue', 'red', 'yellow', 'green', 'brown']
    traces = []
    for i in range(subsample_length):
        # give a color to certain topics
        if 'Apple'.lower() in titles[i].lower():
            color = colors[2]
        elif 'Facebook'.lower() in titles[i].lower() or 'instagram' in titles[i].lower():
            color = colors[0]
        elif 'Marte'.lower() in titles[i].lower() or 'nasa' in titles[i].lower() or 'meteora' in titles[i].lower() or 'pianeta' in titles[i].lower() or 'Ufo' in titles[i]:
            color = colors[1]
        elif 'Samsung'.lower() in titles[i].lower() or 'galaxy' in titles[i].lower():
            color = colors[3]
        else:
            color = colors[4]
        trace = go.Scatter(
            x = [pcomponents[i, 0]], # get first column
            y = [pcomponents[i, 1]], # get second one
            mode = 'markers',
            marker = dict(
                size = 7,
                color = color,
            ),
            text = titles[i] 
        )
        traces.append(trace)
    return traces

In [12]:
traces = []
titles = [doc['title'] for doc in docs]
sub = 100
for model in models:
    trace = visualize_pca(model, 80, titles)
    traces.append(trace)
# visualize one at a time (plotly limitation)
data = traces[2]
layout = dict(title = 'PCA Representantion of DocVectors',
        hovermode= 'closest',
        xaxis= dict(
            title= 'first component',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'second component',
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend = False
    )
# Plot and embed in ipython notebook!
fig = dict(data = data, layout = layout)
py.iplot(fig, filename='pca testing-good')

In [13]:
data = traces[0]
layout = dict(title = 'PCA Representantion of DocVectors',
        hovermode= 'closest',
        xaxis= dict(
            title= 'first component',
            ticklen= 5,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'second component',
            ticklen= 5,
            gridwidth= 2,
        ),
        showlegend = False
    )
# Plot and embed in ipython notebook!
fig = dict(data = data, layout = layout)
py.iplot(fig, filename='pca testing-bad')